Text Representations: Words to Numbers
---

Computers today can not act on words or text directly. They need to be represented by meaningful number sequences. These long sequences of decimal numbers are called vectors. 

Where are these word vectors used?

- Text Classification and Summarization tasks
- Similar words search e.g. synonyms, logically similar
- Machine Translation (e.g Translate text from English to German)
- Understanding Similar texts (e.g. fb feed articles) 
- Question Answering and doing tasks (e.g chatbots in scheduling appointments etc.)

Code Examples
---

Using a machine learning or deep learning model for classification, with following text vectorization methods: 
- TF-IDF in sklearn pipelines with Logistic Regression
- GLove by Stanford, looked up via gensim - we will use pretrained vectors
- fastText by Facebook - using pretrained vectors 

Vectorizing a Specific Dataset
- How to train our own word2vec vectors? 
- How to train our own fastText vectors? 
- Use the `most_similar` words to compare both of the above

Document Embeddings 
- Document modeling using doc2vec on a small dataset
- Using document embedding for clustering

In [1]:
import gensim
print(f'gensim: {gensim.__version__}')

D:\Miniconda3\envs\nlp\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


gensim: 3.4.0


Let's download some pre-trained GLove embeddings: 

In [2]:
# !conda install -y tqdm

In [3]:
from tqdm import tqdm
class TqdmUpTo(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None: self.total = tsize
        self.update(b * bsize - self.n)

def get_data(url, filename):
    """
    Download data if the filename does not exist already
    Uses Tqdm to show download progress
    """
    import os
    from urllib.request import urlretrieve
    
    if not os.path.exists(filename):

        dirname = os.path.dirname(filename)
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        with TqdmUpTo(unit='B', unit_scale=True, miniters=1, desc=url.split('/')[-1]) as t:
            urlretrieve(url, filename, reporthook=t.update_to)

In [4]:
embedding_url = 'http://nlp.stanford.edu/data/glove.6B.zip'
get_data(embedding_url, 'data/glove.6B.zip')

### GLoVe or  word2vec?


In general, I recommend using **GLoVe over word2vec**. This is because it outperforms word2vec on most machine learning and NLP challenges in academia as well as my limited experience. 

I am convinced enough to skip the original word2vec completely here. But for the sake of completeness, we will see the following: 

- How to use the original embeddings? Example: GLoVe
- How to handle Out of Vocabulary words? Hint: FastText
- How to train your own word2vec vectors on your own corpus? 

In [5]:
# # We need to run this only once, can unzip manually unzip to the data directory too
# !unzip data/glove.6B.zip 
# !mv glove.6B.300d.txt data/glove.6B.300d.txt 
# !mv glove.6B.200d.txt data/glove.6B.200d.txt 
# !mv glove.6B.100d.txt data/glove.6B.100d.txt 
# !mv glove.6B.50d.txt data/glove.6B.50d.txt 

### How to use pre-trained embeddings?

**Challenge**: The file formats used by word2vec and GloVe are slightly different from each other. We'd like a consistent API to lookup any word embedding. We can do this 

**Solution**: This format conversion can be done using `gensim`'s API called `glove2word2vec`. We will use this to convert our glove embedding information to word2vec format.

In [6]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'data/glove.6B.300d.txt'

word2vec_output_file = 'data/glove.6B.300d.word2vec.txt'

In [7]:
import os
if not os.path.exists(word2vec_output_file):
    glove2word2vec(glove_input_file, word2vec_output_file)

### KeyedVectors API
We now have the simple task of loading the vectors from a file. We do this using `KeyedVectors` API in gensim. The word we want to lookup is the _key_ and the numerical representation of that word is the corresponding _value_. 

In [8]:
from gensim.models import KeyedVectors
filename = word2vec_output_file 

In [9]:
%%time
# load the Stanford GloVe model from file, this is Disk I/O and can be slow
pretrained_w2v_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
# binary=False format for human readable text (.txt) files, and binary=True for .bin files 

Wall time: 1min 29s


In [10]:
# calculate: (king - man) + woman = ?
result = pretrained_w2v_model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

D:\Miniconda3\envs\nlp\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app


[('queen', 0.6713277101516724)]


In [11]:
# calculate: (india - canada) +  = ?
result = pretrained_w2v_model.most_similar(positive=['quora', 'facebook'], negative=['linkedin'], topn=1)
print(result)

[('twitter', 0.37966805696487427)]


In [12]:
pretrained_w2v_model.most_similar('india')

[('indian', 0.7355823516845703),
 ('pakistan', 0.7285579442977905),
 ('delhi', 0.6846907138824463),
 ('bangladesh', 0.6203191876411438),
 ('lanka', 0.609517514705658),
 ('sri', 0.6011613607406616),
 ('kashmir', 0.5746493935585022),
 ('nepal', 0.5421023368835449),
 ('pradesh', 0.5405811071395874),
 ('maharashtra', 0.518537700176239)]

#### What is missing in both word2vec and GloVe? 
Both glove and word2vec can not handle words or which they did not see during training. These words are called "out of vocabulary" or **OOV** in literature. 

This is evident if you try to lookup nouns which are not frequently used e.g. an uncommon person's name - the model throws a `not in vocabulary` error. 

In [13]:
try:
    pretrained_w2v_model.wv.most_similar('nirant')
except Exception as e:
    print(e)

D:\Miniconda3\envs\nlp\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app


"word 'nirant' not in vocabulary"


### How to handle OOV words? 
The author of word2vec (Mikolov et al.) extended it to create fastText at Facebook. They work on character n-grams instead of the entire words. Character n-grams are effective in languages with specific morphological properties s 

We can create our own fastText embeddings -  which can handle OOV tokens as well

### Get the Dataset

We download the subtitles of several TED talk from a public dataset. We will train our fastText embeddings on these as well as the word2vec embeddings for comparison. 

In [14]:
ted_dataset = "https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip"
get_data(ted_dataset, "data/ted_en.zip")

In [15]:
import zipfile
import lxml.etree
with zipfile.ZipFile('data/ted_en.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))

In [16]:
input_text[:500]

"Here are two reasons companies fail: they only do more of the same, or they only do what's new.\nTo me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation. Both are necessary, but it can be too much of a good thing.\nConsider Facit. I'm actually old enough to remember them. Facit was a fantastic company. They were born deep in the Swedish forest, and they made the best mechanical calculators in the world. Everybody used them. A"

Since we are using subtitles from TED talks, there are some fillers which are not useful. These are often words describing sound in the parenthesis and the speaker’s name. 

Let's remove these fillers: 

In [17]:
import re
# remove parenthesis 
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

# store as list of sentences
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# store as list of lists of words
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Exercise for the reader: 
    Replace the .split() used above with the tokenizer from spacy and see how the `senetences_ted` changes

In [18]:
print(sentences_ted[:2])

[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']]


Notice that each `sentenced_ted` is now a list of list. Each element of the first list is a sentence, and each sentence is a list of tokens (e.g. words).

This is the expected structure for training text embeddings using `gensim`. We will write this to disk for easy retrieval later. 

In [19]:
import json
# with open('ted_clean_sentences.json', 'w') as fp:
#     json.dump(sentences_ted, fp)

In [20]:
with open('ted_clean_sentences.json', 'r') as fp:
    sentences_ted = json.load(fp)

In [21]:
print(sentences_ted[:2])

[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']]


### Train FastText Embedddings

In [22]:
from gensim.models.fasttext import FastText

In [23]:
%%time
fasttext_ted_model = FastText(sentences_ted, size=100, window=5, min_count=5, workers=-1, sg=1)
# sg = 1 denotes skipgram, else CBOW is used

Wall time: 6.17 s


In [24]:
fasttext_ted_model.wv.most_similar("india")

[('indians', 0.5911639928817749),
 ('indian', 0.5406097769737244),
 ('indiana', 0.4898717999458313),
 ('indicated', 0.4400438070297241),
 ('indicate', 0.4042605757713318),
 ('internal', 0.39166826009750366),
 ('interior', 0.3871103823184967),
 ('byproducts', 0.3752930164337158),
 ('princesses', 0.37265270948410034),
 ('indications', 0.369659960269928)]

### Train word2vec Embeddings

In [25]:
from gensim.models.word2vec import Word2Vec

In [26]:
%%time
word2vec_ted_model = Word2Vec(sentences=sentences_ted, size=100, window=5, min_count=5, workers=-1, sg=1)

Wall time: 1.22 s


In [27]:
word2vec_ted_model.wv.most_similar("india")

[('comfortably', 0.3444310426712036),
 ('sargassum', 0.32818296551704407),
 ('cuisine', 0.32582974433898926),
 ('alarms', 0.32521432638168335),
 ('awarded', 0.3224356472492218),
 ('regiment', 0.32133349776268005),
 ('prejudice', 0.31377238035202026),
 ('humanities', 0.31334108114242554),
 ('monotony', 0.3130676746368408),
 ('statements', 0.31284841895103455)]

## fastText or word2vec? 

According to the preliminary comparisons by gensim: 
> fastText embeddings are significantly better than word2vec at encoding syntactic information. This is expected, since most syntactic analogies are morphology based, and the char n-gram approach of fastText takes such information into account. The original word2vec model seems to perform better on semantic tasks, since words in semantic analogies are unrelated to their char n-grams, and the added information from irrelevant char n-grams worsens the embeddings.

> Source: [word2vec fasttext comparison notebook](https://github.com/RaRe-Technologies/gensim/blob/37e49971efa74310b300468a5b3cf531319c6536/docs/notebooks/Word2Vec_FastText_Comparison.ipynb)

In general, prefer fasttext for most web-scale systems because of it's capability to handle words which it has not seen in training. It is definitely better than word2vec on small data (as evident above), and at least as good as word2vec on larger datasets. 

Here is a thumb rule for commercial applications: fastText > GloVe > word2vec. This is obviously not always true, but empirically most common result. 

# Document Embeddings
This section is based on the [Doc2Vec API Tutorial](https://github.com/RaRe-Technologies/gensim/blob/37e49971efa74310b300468a5b3cf531319c6536/docs/notebooks/doc2vec-wikipedia.ipynb) from gensim repository.

In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
from pprint import pprint
import multiprocessing

D:\Miniconda3\envs\nlp\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
import zipfile
import lxml.etree
with zipfile.ZipFile('data/ted_en.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
    
talks = doc.xpath('//content/text()')

In [4]:
def read_corpus(talks, tokens_only=False):
    for i, line in enumerate(talks):
        if tokens_only:
            yield gensim.utils.simple_preprocess(line)
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [6]:
read_corpus(talks)

<generator object read_corpus at 0x0000024741DBA990>

In [7]:
ted_talk_docs = list(read_corpus(talks)) 

In [9]:
ted_talk_docs[0]

TaggedDocument(words=['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 'new', 'to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation', 'both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'good', 'thing', 'consider', 'facit', 'actually', 'old', 'enough', 'to', 'remember', 'them', 'facit', 'was', 'fantastic', 'company', 'they', 'were', 'born', 'deep', 'in', 'the', 'swedish', 'forest', 'and', 'they', 'made', 'the', 'best', 'mechanical', 'calculators', 'in', 'the', 'world', 'everybody', 'used', 'them', 'and', 'what', 'did', 'facit', 'do', 'when', 'the', 'electronic', 'calculator', 'came', 'along', 'they', 'continued', 'doing', 'exactly', 'the', 'same', 'in', 'six', 'months', 'they', 'went', 'from', 'maximum', 'revenue', 'and', 'they', 'were', 'gone'

In [32]:
cores = multiprocessing.cpu_count()
print(cores)

8


In [33]:
model = Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, epochs=5, workers=cores)

In [34]:
%time model.build_vocab(ted_talk_docs)

Wall time: 1.56 s


In [35]:
sentence_1 = 'Modern medicine has changed the way we think about healthcare, life spans and by extension career and marriage'

In [36]:
sentence_2 = 'Modern medicine is not just a boon to the rich, making the raw chemicals behind these is also pollutes the poorest neighborhoods'

In [37]:
sentence_3 = 'Modern medicine has changed the way we think about healthcare, and increased life spans, delaying weddings'

In [38]:
model.docvecs.similarity_unseen_docs(model, sentence_1.split(), sentence_3.split())

-0.18353473068679

In [39]:
model.docvecs.similarity_unseen_docs(model, sentence_1.split(), sentence_2.split())

-0.08177642293252027

In [40]:
%time model.train(ted_talk_docs, total_examples=model.corpus_count, epochs=model.epochs)

Wall time: 6.61 s


In [41]:
model.infer_vector(sentence_1.split())

array([-0.03805782,  0.09805363, -0.07234333,  0.31308332,  0.09668373,
       -0.01471598, -0.16677614, -0.08661497, -0.20852503, -0.14948   ,
       -0.20959479,  0.17605443,  0.15131783, -0.17354141, -0.20173495,
        0.11115499,  0.38531387, -0.39101505,  0.12799   ,  0.0808568 ,
        0.2573657 ,  0.06932276,  0.00427534, -0.26196653,  0.23503092,
        0.07589306, -0.01828301,  0.38289976, -0.04719075, -0.19283117,
        0.1305226 , -0.1426582 , -0.05023642, -0.11381021,  0.04444459,
       -0.04242943,  0.08780348,  0.02872207, -0.23920575,  0.00984556,
        0.0620702 , -0.07004016,  0.15629964,  0.0664391 ,  0.10215732,
        0.19148728, -0.02945088,  0.00786009, -0.05731675, -0.16740018,
       -0.1270729 ,  0.10185472,  0.16655563,  0.13184668,  0.18476236,
       -0.27073956, -0.04078012, -0.12580603,  0.02078131,  0.23821649,
        0.09743162, -0.1095973 , -0.22433399, -0.00453655,  0.29851952,
       -0.21170728,  0.1928157 , -0.06223159, -0.044757  ,  0.02

In [42]:
model.docvecs.similarity_unseen_docs(model, sentence_1.split(), sentence_3.split())

0.9010817740272721

In [43]:
model.docvecs.similarity_unseen_docs(model, sentence_1.split(), sentence_2.split())

0.7461058869759862

In [44]:
model.docvecs.similarity_unseen_docs(model, sentence_2.split(), sentence_3.split())

0.8189999598358203

# Model Assessment

One simple technique to assess any vectorization method is to simply use the training corpus again as the test corpus. Of course, we expect that we will overfit our model to training set, but that's fine. 

We use the train corpus as test corpus by doing the following:
- Learn new result or 'inference' vectors for each document
- Compared the vector to all examples
- Rank the document/sentence/paragraph vectors according to the similarity score

In [45]:
ranks = []
for idx in range(len(ted_talk_docs)):
    inferred_vector = model.infer_vector(ted_talk_docs[idx].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(idx)
    ranks.append(rank)

In [46]:
import collections
collections.Counter(ranks)  # Results vary due to random seeding + very small corpus

Counter({0: 2079, 1: 2, 4: 1, 5: 2, 2: 1})

In [47]:
doc_slice = ' '.join(ted_talk_docs[idx].words)[:500]
print(f'Document ({idx}): «{doc_slice}»\n')
print(f'SIMILAR/DISSIMILAR DOCS PER MODEL {model}')
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
      doc_slice = ' '.join(ted_talk_docs[sims[index][0]].words)[:500]
      print(f'{label} {sims[index]}: «{doc_slice}»\n')

Document (2084): «if you re here today and very happy that you are you ve all heard about how sustainable development will save us from ourselves however when we re not at ted we are often told that real sustainability policy agenda is just not feasible especially in large urban areas like new york city and that because most people with decision making powers in both the public and the private sector really don feel as though they re in danger the reason why here today in part is because of dog an abandoned puppy»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dbow,d100,n5,mc2,s0.001,t8)
MOST (2084, 0.893369197845459): «if you re here today and very happy that you are you ve all heard about how sustainable development will save us from ourselves however when we re not at ted we are often told that real sustainability policy agenda is just not feasible especially in large urban areas like new york city and that because most people with decision making powers in both the public and the priva